### GPU solver for Aguirregabria model

In [1]:
using FastGaussQuadrature
using GPUArrays, CLArrays
include("../julia/aguirregabiria_core.jl")

# Initializing the OpenCL device & context:
cl_device = CLArrays.devices()[1]
CLArrays.init(cl_device)

OpenCL context with:
CL version: OpenCL 2.1 NEO 
Device: CL Intel(R) UHD Graphics 620
            threads: 256
             blocks: (256, 256, 256)
      global_memory: 3352.154112 mb
 free_global_memory: NaN mb
       local_memory: 0.065536 mb


In [ ]:
# Control GPU kernel

gpu_A = ones(CLArray{Float32}, 10, 1)

gpu_call(gpu_A, (gpu_A,)) do state, gpu_A
  begin
     
        function _add(a::Float32, b::Float32)
            return (a + b)*2
        end
        
        i = @linearidx gpu_A
        
        gpu_A[i] = _add(Float32(1.0), Float32(2.0))  
        
        return
  end
end

@time gpu_A;

In [8]:
lambdas      = generate_simplex_3dims(5)
n_price_grid = 20
min_price    = 0.5
max_price    = 1.5
price_grid   = linspace(min_price, max_price, n_price_grid)

function V_0(x) 
    optimal_price = myopic_price(x)
    return period_return(optimal_price, x)
end

V_0 (generic function with 1 method)

In [11]:
# GPU kernel for Bellman operator:

x, w = gausshermite(25)
    
GHx = CLArray{Float32}(x)
GHw = CLArray{Float32}(w)

policy = zeros(CLArray{Float32}, size(lambdas, 1))
V_j    = zeros(CLArray{Float32}, size(lambdas, 1))

function gpu_bellman_operator(V::T, Vnew::T, policy::T, price_grid::T, lambda_simplex::T, params::T) where T<:GPUArray
    # GPU kernel for Bellman operator
    # - Vguess, price_grid, lambda_simplex are onlready GPUArrays
    # - 
    
    gpu_call(V, (V, Vnew, policy, price_grid, lambda_simplex, params)) do state, V, Vnew, policy, price_grid, lambda_simplex, params
      begin
            
        # {utility functions}
        
        function tripolate(V, x::Float32, y::Float32)
            # Triangular (linear) interpolation:
            # 0<=|x|<=1, 0<=|y|<=1 and N parition points
            # of [0,1] for x, y
            N = size(V, 1)-1
            xf = Int8(floor(N*x))
            yf = Int8(floor(N*y))
            x_ = (N*x - xf)
            y_ = (N*y - yf)
            if xf + yf <= 1
                return V[xf+2,yf+1]*x_ + V[xf+1,yf+2]*y_ + V[xf+1,yf+1]*(1-x_-y_)
            else
                return V[xf+2,yf+1]*(1-x_) + V[xf+1,yf+2]*(1-y_) - V[xf+1,yf+1]*(1-x_-y_)
            end 
        end
                
        # {kernel code}
        
        i = @linearidx V
        
        gpu_A[i] = i 
        
        return
      end
    end
    
end

gpu_bellman_operator (generic function with 2 methods)

In [6]:
CLArray<:GPUArray

true